In [ ]:
# ===== 1. Сносим не-подходящие версии (если ставились ранее) =====
# %pip uninstall -y torch torchvision torchaudio torchtext torchdata pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv || true

# ===== 2. Ставим PyTorch 2.4.1 (стабильная версия, совместимая с большинством зависимостей) =====
%pip install -q torch==2.4.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# ===== 3. Ставим PyTorch Geometric 2.2.0 и зависимые колёса для совместимости =====
%pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv
#%pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.4.1+cpu.html
%pip install -q torch-geometric==2.2.0

# ===== 4. Утилиты для проекта =====
%pip install -q pandas scikit-learn networkx matplotlib rich tqdm
%pip install -q graphlime
%pip install focal-loss-torch


In [ ]:
import os, random, numpy as np
import datetime
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, recall_score, confusion_matrix
import torch.optim as optim
from torch_geometric.utils import add_self_loops
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import networkx as nx
from torch_geometric.explain.algorithm.gnn_explainer import GNNExplainer_
import joblib
from collections import OrderedDict
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
# print("Python           :", sys.version.split()[0])
# print("Torch            :", torch.__version__, "| CUDA:", torch.version.cuda, "| GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
# print("PyG              :", torch_geometric.__version__)
# print("torch_scatter    :", torch_scatter.__version__)
# print("pandas           :", pd.__version__)
# print("scikit-learn     :", sklearn.__version__)
# print("networkx         :", nx.__version__)

In [ ]:
#@title 📂 Загрузка датасета (идемпотентно)
import os

# Папка, куда будет сохранён репозиторий
data_dir = 'data/'

# Если папка не существует, создаем её
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Клонирование репозитория в папку data
!git clone https://github.com/salam-ammari/Labeled-Transactions-based-Dataset-of-Ethereum-Network.git {data_dir}


import zipfile, pathlib
zip_path = "data/Dataset.zip"
extract_path = pathlib.Path("data/unpacked")
with zipfile.ZipFile(zip_path) as zf: zf.extractall(extract_path)
print("✅ Распаковка завершена")

In [ ]:
#@title 🧹 Чтение CSV
import pandas as pd
dataset_csv_path = extract_path / "Dataset" / "Dataset.csv"
df = pd.read_csv(dataset_csv_path)
print(df.shape, "строк")
print(df.columns.tolist())

In [ ]:
# @title 🧹 Этап 3 — очистка данных (актуальная версия)

# --- 0. резервная копия ---
df_raw = df.copy()

# --- 1. числовые столбцы ---
num_cols = [
    "nonce", "transaction_index", "value", "gas", "gas_price",
    "receipt_cumulative_gas_used", "receipt_gas_used", "block_number"
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce", downcast="integer")

# --- 2. флаги мошенничества ---
df["from_scam"] = df["from_scam"].astype("int8")
df["to_scam"]   = df["to_scam"].astype("int8")

# --- 3. корректный разбор даты/времени блока ---
# ❶ заменяем ' UTC' → '+00:00' (делает формат единообразным)
ts_fixed = df["block_timestamp"].astype(str).str.replace(" UTC", "+00:00", regex=False)

# ❷ один вызов to_datetime, pandas ≥ 2.0 умеет format='mixed'
df["block_timestamp"] = pd.to_datetime(ts_fixed, utc=True, format="mixed", errors="raise")

# --- 4. удаляем записи без адресов ---
df.dropna(subset=["from_address", "to_address"], inplace=True)

# --- 5. убираем дубликаты по hash ---
dup_before = len(df)
df.drop_duplicates(subset="hash", inplace=True)
dup_removed = dup_before - len(df)

# --- 6. сброс индекса, сохранив исходный ---
df.reset_index(names="raw_row", inplace=True)

# --------- отладочный вывод ----------
print(f"➤ строк после очистки: {len(df)}  (исходно {len(df_raw)})")
print(f"   удалено дубликатов: {dup_removed}")
print("\nМетки транзакций:")
print("  from_scam =", dict(df['from_scam'].value_counts()))
print("  to_scam   =", dict(df['to_scam'].value_counts()))
print("\nТипы данных:")
print(df.dtypes.value_counts())
print("\nПервые 3 строки:")
display(df.head(3))

In [ ]:

# @title Инженерия признаков

# ---------- 1. вспом-функции ----------
def log1p_clip(col):           # логарифм со сдвигом
    return np.log1p(col).astype("float32")

# ---------- 2. агрегация по адресам ----------
g_from = (df
          .groupby("from_address")
          .agg(tx_out_count = ("hash",  "size"),
               tx_out_value = ("value", "sum"),
               first_out_ts = ("block_timestamp", "min"),
               last_out_ts  = ("block_timestamp", "max"))
         )

g_to = (df
        .groupby("to_address")
        .agg(tx_in_count = ("hash",  "size"),
             tx_in_value = ("value", "sum"),
             first_in_ts = ("block_timestamp", "min"),
             last_in_ts  = ("block_timestamp", "max"))
       )

nodes_df = g_from.join(g_to, how="outer")

# ---------- 3. заполняем пропуски адресно ----------
num_cols  = ["tx_out_count","tx_out_value","tx_in_count","tx_in_value"]
ts_cols   = ["first_out_ts","last_out_ts","first_in_ts","last_in_ts"]

nodes_df[num_cols] = nodes_df[num_cols].fillna(0).astype({"tx_out_count":"int32",
                                                          "tx_in_count":"int32"})
# временные оставляем как NaT – с ними корректно работают min/max

# ---------- 4. years_active ----------
first_ts = nodes_df[["first_out_ts","first_in_ts"]].min(axis=1, skipna=True)
last_ts  = nodes_df[["last_out_ts","last_in_ts"]].max(axis=1, skipna=True)

years = (last_ts - first_ts).dt.total_seconds() / (365*24*3600)
nodes_df["years_active"] = years.fillna(0).astype("float32")

# ---------- 5. логарифмируем суммы ----------
for col in ["tx_out_value","tx_in_value"]:
    nodes_df[col] = log1p_clip(nodes_df[col])

# ---------- 6. метка scam-узла ----------
scam_mask = pd.Series(0, index=nodes_df.index, dtype="int8")
scam_addr = pd.unique(pd.concat([df.loc[df["from_scam"]==1,"from_address"],
                                 df.loc[df["to_scam"]==1,"to_address"]]))
scam_mask.loc[scam_addr] = 1
nodes_df["is_scam"] = scam_mask

# ---------- 7. таблица рёбер ----------
edges_df = df[["from_address","to_address","value","block_timestamp"]].copy()
edges_df.rename(columns={"from_address":"source",
                         "to_address":"target"}, inplace=True)
edges_df["log_value"] = log1p_clip(edges_df["value"])
edges_df["ts_norm"]   = edges_df["block_timestamp"].astype("int64") / 1e9
edges_df.drop(columns=["value","block_timestamp"], inplace=True)

# ---------- 8. отладка ----------
print(f"Nodes : {nodes_df.shape}")
print(f"Edges : {edges_df.shape}")
print("\nSample nodes_df:")
display(nodes_df.head(3))
print("\nSample edges_df:")
display(edges_df.head(3))
print("\nlabel distribution:")
print(nodes_df['is_scam'].value_counts())

In [ ]:
# @title # Загрузка и подготовка данных
addr2id = {addr: i for i, addr in enumerate(nodes_df.index)}
num_nodes = len(addr2id)
src = edges_df["source"].map(addr2id).to_numpy()
dst = edges_df["target"].map(addr2id).to_numpy()
edge_index = torch.tensor(np.vstack([src, dst]), dtype=torch.long)
edge_attr = torch.tensor(edges_df[["log_value", "ts_norm"]].to_numpy(dtype=np.float32), dtype=torch.float32)
x = torch.tensor(nodes_df[["tx_out_count", "tx_out_value", "tx_in_count", "tx_in_value", "years_active"]].to_numpy(dtype=np.float32), dtype=torch.float32)
y = torch.tensor(nodes_df["is_scam"].to_numpy(), dtype=torch.long)

In [ ]:
# @title # Разбиение на train, val, test
train_idx, temp_idx = train_test_split(np.arange(num_nodes), test_size=0.5, stratify=y, random_state=SEED)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=y[temp_idx], random_state=SEED)
def to_mask(idxs, size):
    m = torch.zeros(size, dtype=torch.bool)
    m[idxs] = True
    return m

train_mask = to_mask(train_idx, num_nodes)
val_mask = to_mask(val_idx, num_nodes)
test_mask = to_mask(test_idx, num_nodes)

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

In [ ]:
# @title # Модель GNN
class GNNModel(torch.nn.Module):
    def __init__(self, in_feats):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_feats, 64)
        self.conv2 = GCNConv(64, 32)
        self.conv3 = GCNConv(32, 16)
        self.conv4 = GCNConv(16, 2)  # Output layer

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.dropout(x, p=0.2, train=self.training)  # Dropout для регуляризации
        x = torch.relu(self.conv2(x, edge_index))
        x = torch.dropout(x, p=0.2, train=self.training)
        x = torch.relu(self.conv3(x, edge_index))
        x = self.conv4(x, edge_index)  # Output logits
        return x

In [ ]:
# Инициализация модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GNNModel(in_feats=data.x.size(1)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [ ]:

# Функция потерь
class_weights = torch.tensor([1.0, 10.0], device=device)  # Увеличиваем вес для класса мошенников
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
# Обучение модели с ранней остановкой
EPOCHS = 200
best_rec = 0.0
no_improve = 0
best_state = None

for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        out_val = model(data.x, data.edge_index)
        val_loss = criterion(out_val[data.val_mask], data.y[data.val_mask]).item()
        val_pred = out_val[data.val_mask].argmax(dim=1).cpu()
        val_true = data.y[data.val_mask].cpu()
        val_rec = recall_score(val_true, val_pred, zero_division=0)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss:.4f} | Val Recall: {val_rec:.4f}")

    if val_rec > best_rec:
        best_rec = val_rec
        best_state = model.state_dict()
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= 3:
            print("Early stopping triggered.")
            break

In [ ]:
# Оценка на тестовой выборке
model.load_state_dict(best_state)
model.eval()
with torch.no_grad():
    out_test = model(data.x, data.edge_index)
    test_pred = out_test[data.test_mask].argmax(dim=1).cpu()
    test_true = data.y[data.test_mask].cpu()
    test_rec = recall_score(test_true, test_pred, zero_division=0)
    cm = confusion_matrix(test_true, test_pred)

print(f"Test Recall: {test_rec:.4f}")
print(f"Confusion Matrix:\n{cm}")

In [ ]:
# Визуализация: ROC и Precision-Recall
fpr, tpr, _ = roc_curve(test_true, out_test[data.test_mask].softmax(dim=1)[:, 1].cpu())
roc_auc = auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(test_true, out_test[data.test_mask].softmax(dim=1)[:, 1].cpu())

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], '--', lw=1)
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.subplot(1, 2, 2)
plt.plot(recall, precision, lw=2)
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.tight_layout()
plt.show()



In [ ]:
import os
import torch
import pickle
from pathlib import Path
import numpy as np

# Создаём папку для артефактов
artifacts_dir = Path("artifacts")
artifacts_dir.mkdir(exist_ok=True)

# Сохраняем модель (включая архитектуру и веса)
model_path = artifacts_dir / "gnn_eth_fraud_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Модель сохранена в {model_path}")

# Сохраняем список признаков (x_cols)
x_cols = ["tx_out_count", "tx_out_value", "tx_in_count", "tx_in_value", "years_active"]  # Признаки, которые используются в модели
features_path = artifacts_dir / "features_list.pkl"
with open(features_path, "wb") as f:
    pickle.dump(x_cols, f)
print(f"Список признаков сохранён в {features_path}")

# Если у тебя есть преобразователь признаков (например, scaler для нормализации),
# ты можешь сохранить его так же, как это делается в LSTM-коде.

# Пример для Scaler (если используется):
from sklearn.preprocessing import MinMaxScaler

# Пример скейлера
scaler = MinMaxScaler()
scaler.fit(x.numpy())  # Если твои данные были в numpy массиве
scaler_path = artifacts_dir / "scaler.pkl"
with open(scaler_path, "wb") as f:
    pickle.dump(scaler, f)
print(f"Scaler сохранён в {scaler_path}")

# Создание директории для чекпоинтов
checkpoint_dir = artifacts_dir / "checkpoints"
checkpoint_dir.mkdir(exist_ok=True)

# Путь для сохранения лучших весов (чекпоинтов)
checkpoint_path = checkpoint_dir / "gnn_best_weights.pth"

# Сохранение модели с лучшими весами
torch.save(model.state_dict(), checkpoint_path)
print(f"Лучшие веса сохранены в {checkpoint_path}")

# Сохранение эмбеддингов (если необходимо)
# Для GNN эмбеддинги могут быть извлечены


In [ ]:
# Этап 8 — Использование GNNExplainer
explainer = GNNExplainer_(
    model,
    epochs=200,
    lr=0.01,
    return_type='log_prob',       # лог-вероятности
    feat_mask_type='feature',     # маскируем признаки
    allow_edge_mask=True,         # маскируем рёбра
)

# Выбираем узел для объяснения (первый мошенник)
node_idx = (data.y == 1).nonzero(as_tuple=False)[0].item()

# Получаем маски важности:
node_feat_mask, edge_mask = explainer.explain_node(
    node_idx,
    x=data.x,
    edge_index=data.edge_index,
)

# Визуализируем важные рёбра
threshold = float(edge_mask.mean().cpu())
G = nx.Graph()
src, dst = data.edge_index.cpu().numpy()
for i, (u, v) in enumerate(zip(src, dst)):
    if edge_mask[i] > threshold:
        G.add_edge(int(u), int(v), weight=float(edge_mask[i].cpu()))

plt.figure(figsize=(6, 6))
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_size=30)
nx.draw_networkx_edges(
    G, pos,
    width=[d['weight']*5 for (_, _, d) in G.edges(data=True)],
    alpha=0.8
)
plt.title(f"Important subgraph for node {node_idx}")
plt.axis('off')
plt.show()

x_cols = ["tx_out_count", "tx_out_value", "tx_in_count", "tx_in_value", "years_active"]


# Печатаем важность признаков для этого узла
print("Feature importances:")
for feat_name, score in zip(x_cols, node_feat_mask.cpu()):
    print(f"  {feat_name}: {score:.4f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
import joblib, pandas as pd
from collections import OrderedDict
import torch

# Подготовка данных для сравнения
X = nodes_df[["tx_out_count", "tx_out_value", "tx_in_count", "tx_in_value", "years_active"]].values
y = nodes_df["is_scam"].values
train_idx = data.train_mask.cpu().numpy()  # Индексы для тренировочной выборки
test_idx = data.test_mask.cpu().numpy()  # Индексы для тестовой выборки

# Определяем базовые модели
models = OrderedDict({
    "RF": RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42),
    "DT": DecisionTreeClassifier(class_weight="balanced", random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5, weights="distance")
})

# Список для хранения результатов
scores = []

# Обучаем и оцениваем модели
for name, clf in models.items():
    clf.fit(X[train_idx], y[train_idx])  # Обучаем модель
    y_hat = clf.predict(X[test_idx])  # Делаем предсказания
    p, r, f, _ = precision_recall_fscore_support(
        y[test_idx], y_hat, zero_division=0, average="binary"
    )  # Оценка precision, recall, F1
    scores.append((name, p, r, f))
    print(f"{name:3}  precision {p:.3f}  recall {r:.3f}  F1 {f:.3f}")

# Создаем DataFrame для результатов
baseline_df = pd.DataFrame(scores, columns=["model", "precision", "recall", "F1"])
print("\nBaseline Models Comparison:")
print(baseline_df)

# Теперь оценим модель GNN
model.eval()
with torch.no_grad():
    out_test = model(data.x, data.edge_index)  # Получаем предсказания GNN
    test_pred = out_test[data.test_mask].argmax(dim=1).cpu()  # Преобразуем в классы
    test_true = data.y[data.test_mask].cpu()  # Истинные значения
    test_p, test_r, test_f, _ = precision_recall_fscore_support(
        test_true, test_pred, zero_division=0, average="binary"
    )  # Оценка модели GNN

# Добавляем результаты модели GNN в таблицу
gnn_results = {
    "model": "GNN",
    "precision": test_p,
    "recall": test_r,
    "F1": test_f
}

# Используем pd.concat вместо append
gnn_results_df = pd.DataFrame([gnn_results])  # Преобразуем в DataFrame
baseline_df = pd.concat([baseline_df, gnn_results_df], ignore_index=True)

# Выводим окончательные результаты
print("\nComparison with GNN:")
print(baseline_df)
